In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd

In [2]:
df = pd.read_csv("../data/clean_data.csv")
X = df.drop(columns=['Depression'])
y = df['Depression']
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


numeric_cols = ['Age', 'Academic Pressure', 'CGPA', 'Study Satisfaction', 'Study Hours', 'Financial Stress']
categorical_cols = ['Gender', 'City', 'Degree', 'Dietary Habits', 'Sleep Duration']

preprocessor_lr = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])


In [3]:
X_train_scaled = preprocessor_lr.fit_transform(X_train)
X_test_scaled = preprocessor_lr.transform(X_test)

In [4]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


C:\Users\nakul\anaconda3\envs\depression\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_scaled, y_train, epochs=7, batch_size=42, validation_split=0.2, verbose=1)

Epoch 1/7
425/425 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7817 - loss: 0.4679 - val_accuracy: 0.8074 - val_loss: 0.4280
Epoch 2/7
425/425 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7964 - loss: 0.4440 - val_accuracy: 0.8061 - val_loss: 0.4309
Epoch 3/7
425/425 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8005 - loss: 0.4382 - val_accuracy: 0.8036 - val_loss: 0.4337
Epoch 4/7
425/425 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8020 - loss: 0.4355 - val_accuracy: 0.8054 - val_loss: 0.4284
Epoch 5/7
425/425 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8055 - loss: 0.4310 - val_accuracy: 0.8043 - val_loss: 0.4304
Epoch 6/7
425/425 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8063 - loss: 0.4297 - val_accuracy: 0.8012 - val_loss: 0.4321
Epoch 7/7
425/425 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8068 - loss: 0.4239 - val_accuracy: 0.8052 - val_loss: 0.4323


In [6]:
y_pred_proba = model.predict(X_test_scaled).ravel()
y_pred = (y_pred_proba > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_proba))


175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.7955890263582571
ROC-AUC: 0.8641426570068917
